https://www.londonair.org.uk/LondonAir/API/

Docs http://api.erg.kcl.ac.uk/AirQuality/Information/Documentation/pdf

In [1]:
import requests
import json
import pprint

def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

# Single site - Hourly

In [2]:
SiteCode = 'KT4'
url = 'http://api.erg.kcl.ac.uk/AirQuality/Hourly/MonitoringIndex/SiteCode={}/Json'.format(SiteCode)
url

'http://api.erg.kcl.ac.uk/AirQuality/Hourly/MonitoringIndex/SiteCode=KT4/Json'

In [3]:
response = requests.get(url).json()  # get the response object

In [4]:
print_json(response)

{'HourlyAirQualityIndex': {'@TimeToLive': '33',
                           'LocalAuthority': {'@LaCentreLatitude': '51.412317',
                                              '@LaCentreLatitudeWGS84': '6694554.466476',
                                              '@LaCentreLongitude': '-0.300443',
                                              '@LaCentreLongitudeWGS84': '-33445.161772',
                                              '@LocalAuthorityCode': '21',
                                              '@LocalAuthorityName': 'Kingston',
                                              'Site': {'@BulletinDate': '2017-07-30 '
                                                                        '19:00:00',
                                                       '@Latitude': '51.379312',
                                                       '@LatitudeWGS84': '6688665.88821522',
                                                       '@Longitude': '-0.281259',
                             

In [5]:
base = response['HourlyAirQualityIndex']['LocalAuthority']['Site']

In [6]:
base['@SiteName']

'Kingston Upon Thames - Tolworth Broadway'

In [7]:
for species in base['species']:
    print('*****')
    print(species['@SpeciesName'])
    print(species['@AirQualityIndex'])
    print(species['@AirQualityBand'])

*****
Nitrogen Dioxide
1
Low
*****
PM10 Particulate
1
Low


# All sites - Hourly

In [8]:
url = 'http://api.erg.kcl.ac.uk/AirQuality/Hourly/MonitoringIndex/GroupName=London/Json'
response = requests.get(url).json()  # get the response object

In [9]:
base = response['HourlyAirQualityIndex']['LocalAuthority']
print(len(base))   # 33 sites
autorities_with_site = []
authorities = []

for authority in base:
    authorities.append(authority['@LocalAuthorityName'])
    if 'Site' in authority.keys():
        #print(authority['@LocalAuthorityName'])
        autorities_with_site.append(authority['@LocalAuthorityName'])
        
print(len(autorities_with_site))
print(list(set(set(authorities) - set(autorities_with_site))))

33
28
['Bromley', 'Newham', 'Hounslow', 'Waltham Forest', 'Barnet']


In [10]:
data = {}

for authority in base:
    if authority['@LocalAuthorityName'] == 'Kingston':
        data = authority
        
data

{'@LaCentreLatitude': '51.412317',
 '@LaCentreLatitudeWGS84': '6694554.466476',
 '@LaCentreLongitude': '-0.300443',
 '@LaCentreLongitudeWGS84': '-33445.161772',
 '@LocalAuthorityCode': '21',
 '@LocalAuthorityName': 'Kingston',
 'Site': {'@BulletinDate': '2017-07-30 19:00:00',
  '@Latitude': '51.379312',
  '@LatitudeWGS84': '6688665.88821522',
  '@Longitude': '-0.281259',
  '@LongitudeWGS84': '-31309.6086610253',
  '@SiteCode': 'KT4',
  '@SiteName': 'Kingston Upon Thames - Tolworth Broadway',
  '@SiteType': 'Roadside',
  'Species': [{'@AirQualityBand': 'Low',
    '@AirQualityIndex': '1',
    '@IndexSource': 'Measurement',
    '@SpeciesCode': 'NO2',
    '@SpeciesDescription': 'Nitrogen Dioxide'},
   {'@AirQualityBand': 'Low',
    '@AirQualityIndex': '1',
    '@IndexSource': 'Trigger',
    '@SpeciesCode': 'PM10',
    '@SpeciesDescription': 'PM10 Particulate'}]}}

In [11]:
# If only 1 site, then a dict is returned so wrap in a list to ensure a list ia parsed in next step
if isinstance(data['Site'], dict):
    site_data = [data['Site']]  
else:
    site_data = data['Site']  
    for site in site_data:
        print(site['@SiteName'])
        if site['Species']['@AirQualityBand'] != 'No data':      # don't print if no data
            print(site['Species']['@AirQualityBand'])
            print(site['Species']['@AirQualityIndex'])
            print(site['Species']['@SpeciesDescription'])

In [12]:
print(len(base[2]['Site']))   # there are 8 sites
for site in base[2]['Site']:
    print(site['@SiteName'])

8
Bexley - Belvedere West
Bexley - Belvedere West FDMS
Bexley - Belvedere FDMS
Bexley - Slade Green
Bexley - Belvedere
Bexley - Slade Green FDMS
Greenwich and Bexley - Falconwood FDMS
Greenwich and Bexley - Falconwood


# HASS class

In [13]:
LOCATIONS = 'Locations'
URL = 'http://api.erg.kcl.ac.uk/AirQuality/Hourly/MonitoringIndex/GroupName=London/Json'
#SCAN_INTERVAL = timedelta(seconds=30)

# Remove authorities without any Sites ['Hounslow', 'Bromley', 'Waltham Forest', 'Barnet', 'Newham']
AUTHORITIES = [
    'Barking and Dagenham',
    'Bexley',
    'Brent',
    'Camden',
    'City of London',
    'Croydon',
    'Ealing',
    'Enfield',
    'Greenwich',
    'Hackney',
    'Hammersmith and Fulham',
    'Haringey',
    'Harrow',
    'Havering',
    'Hillingdon',
    'Islington',
    'Kensington and Chelsea',
    'Kingston',
    'Lambeth',
    'Lewisham',
    'Merton',
    'Redbridge',
    'Richmond',
    'Southwark',
    'Sutton',
    'Tower Hamlets',
    'Wandsworth',
    'Westminster']

CONFIG = {LOCATIONS: ['Merton','Richmond',]}  # A list of required locations

In [14]:
def parse_api_response(response):
    """Take in the API response. API can return dict or list of data so need to check. """
    data = dict.fromkeys(AUTHORITIES)     # Holds all data
    for authority in AUTHORITIES:
        for entry in response['HourlyAirQualityIndex']['LocalAuthority']:   # Loop over entries
            if entry['@LocalAuthorityName'] == authority:
                authority_data = []
                
                if isinstance(entry['Site'], dict):
                    entry_sites_data = [entry['Site']]  
                else:
                    entry_sites_data = entry['Site']  
                
                for site in entry_sites_data:
                    site_data = {}
                    species_data = []
                    
                    site_data['Site code'] = site['@SiteCode']
                    site_data['Site name'] = site['@SiteName']  
                    
                    if isinstance(site['Species'], dict):
                        species_data = [site['Species']]  
                    else:
                        species_data = site['Species'] 
                    
                    parsed_species_data = []
                    for species in species_data:
                        if species['@AirQualityBand'] != 'No data': 
                            species_dict = {}
                            species_dict['Description'] = species['@SpeciesDescription']
                            species_dict['Code'] = species['@SpeciesCode']
                            species_dict['Quality'] = species['@AirQualityBand']
                            species_dict['Index'] = species['@AirQualityIndex']
                            parsed_species_data.append(species_dict)
                    
                    if not parsed_species_data:      # if no valid species data
                        parsed_species_data.append('No species data')

                    site_data['Species'] = parsed_species_data
                    authority_data.append(site_data)
                    
                data[authority] = authority_data

    return data

In [15]:
response = requests.get(URL).json()
my_data = parse_api_response(response)

In [16]:
my_data.keys()

dict_keys(['Hackney', 'Camden', 'Greenwich', 'Tower Hamlets', 'Lambeth', 'Havering', 'Kingston', 'Southwark', 'Kensington and Chelsea', 'Hillingdon', 'Sutton', 'Redbridge', 'Lewisham', 'Croydon', 'Richmond', 'Bexley', 'Haringey', 'Hammersmith and Fulham', 'Wandsworth', 'Brent', 'City of London', 'Barking and Dagenham', 'Enfield', 'Westminster', 'Harrow', 'Islington', 'Ealing', 'Merton'])

In [17]:
my_data['Merton']

[{'Site code': 'ME2',
  'Site name': 'Merton - Merton Road',
  'Species': [{'Code': 'PM10',
    'Description': 'PM10 Particulate',
    'Index': '2',
    'Quality': 'Low'}]},
 {'Site code': 'ME9',
  'Site name': 'Merton - Morden Civic Centre 2',
  'Species': [{'Code': 'NO2',
    'Description': 'Nitrogen Dioxide',
    'Index': '1',
    'Quality': 'Low'}]}]

In [18]:
print_json(my_data)

{'Barking and Dagenham': [{'Site code': 'BG1',
                           'Site name': 'Barking and Dagenham - Rush Green',
                           'Species': [{'Code': 'NO2',
                                        'Description': 'Nitrogen Dioxide',
                                        'Index': '1',
                                        'Quality': 'Low'},
                                       {'Code': 'SO2',
                                        'Description': 'Sulphur Dioxide',
                                        'Index': '1',
                                        'Quality': 'Low'}]},
                          {'Site code': 'BG2',
                           'Site name': 'Barking and Dagenham - Scrattons Farm',
                           'Species': [{'Code': 'NO2',
                                        'Description': 'Nitrogen Dioxide',
                                        'Index': '1',
                                        'Quality': 'Low'},
                  

In [19]:
class AirData(object):
    """Get the latest data."""

    def __init__(self):
        """Initialize the AirData object."""
        self.data = None

    def update(self):
        """Get the latest data from TFL."""
        response = requests.get(URL)
        if response.status_code != 200:
            _LOGGER.warning("Invalid response from API")
        else:
            self.data = parse_api_response(response.json())

In [20]:
# Class will return dict of attributes of form

attributes = {'LocalAuthorityName': 'Kingston', 
              'SiteCode': 'KT4',
              'SiteName': 'Kingston upon thames',
              '@BulletinDate': '2017-07-04 03:00:00',              # Will be sensor state
              'species': [
        {'@AirQualityBand': 'Low',
         '@AirQualityIndex': '1',
         '@IndexSource': 'Trigger',
         '@SpeciesCode': 'PM10',
         '@SpeciesName': 'PM10 Particulate'}
    ]}